In [1]:
import polars as pl
import numpy as np
import random
from datetime import datetime, timedelta

# Descripcion

Los datos son notificaciones de dispositivos GPS en Mexico. En promedio generan notificaciones automatizadas cada 5 minutos si el carro esta encendido, y 30 si esta apagado.  

Cada notificacion esta acompannada de un evento de lo que esta ocurriendo, y trae la latitud y longitud.  

El objetico es predecir si un vehiculo esta siendo robado de acuerdo a sus notificaciones, por lo que el primer paso seria limpiar datos y hacer ingenieria de variables.

Trata de hacerlo **lazy** si puedes.

In [32]:
def generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval):
    data = []

    # Define the latitude and longitude ranges for Mexico
    min_latitude, max_latitude = 14.5388, 32.7186
    min_longitude, max_longitude = -118.4662, -86.7104

    for car_id in range(num_cars):
        current_time = start_time

        # Generate random initial latitude and longitude for each car
        latitude = random.uniform(min_latitude, max_latitude)
        longitude = random.uniform(min_longitude, max_longitude)

        while current_time < end_time:
            if current_time.weekday() < 5 and 9 <= current_time.hour < 17:
                # Working hours (Monday to Friday, 9 AM to 5 PM)
                interval = working_hours_interval
            else:
                # Non-working hours
                interval = non_working_hours_interval

            # Generate notification with 99% probability
            if random.random() < 0.99:
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))

            # Generate additional notifications between intervals
            while True:
                additional_interval = random.expovariate(1 / (interval / 2))
                additional_time = current_time + timedelta(minutes=additional_interval)
                if additional_time >= current_time + timedelta(minutes=interval):
                    break
                notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                data.append((f"car_{car_id}", additional_time.isoformat(), latitude, longitude, notification))

            # Update latitude and longitude for car movement
            latitude += random.uniform(-0.01, 0.01)
            longitude += random.uniform(-0.01, 0.01)

            # Check if the car is among the 1% that can have 100 notifications within 5 minutes
            if random.random() < 0.01:
                burst_start_time = current_time + timedelta(minutes=random.uniform(0, interval))
                burst_end_time = burst_start_time + timedelta(minutes=5)
                while current_time < burst_end_time:
                    notification = random.choice(["low_fuel", "tire_pressure", "engine_check", None])
                    data.append((f"car_{car_id}", current_time.isoformat(), latitude, longitude, notification))
                    current_time += timedelta(seconds=random.uniform(1, 10))

            current_time += timedelta(minutes=interval)

    # Create a Polars DataFrame from the generated data
    df = pl.DataFrame(
        {
            "car_id": [record[0] for record in data],
            "timestamp": [record[1] for record in data],
            "latitude": [record[2] for record in data],
            "longitude": [record[3] for record in data],
            "notification": [record[4] for record in data],
        }
    )

    return df.lazy()

In [51]:
num_cars = 1000
start_time = datetime(2023, 1, 1, 0, 0, 0)  # Start of the week
end_time = start_time + timedelta(weeks=1)  # End of the week
working_hours_interval = 5  # Interval of 5 minutes during working hours
non_working_hours_interval = 30  # Interval of 30 minutes during non-working hours

# Generate the dummy data
data = generate_dummy_data(num_cars, start_time, end_time, working_hours_interval, non_working_hours_interval)

# Print the first few rows of the generated data
print(data.head().collect())

shape: (5, 5)
┌────────┬────────────────────────────┬───────────┬────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude  ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---        ┆ ---           │
│ str    ┆ str                        ┆ f64       ┆ f64        ┆ str           │
╞════════╪════════════════════════════╪═══════════╪════════════╪═══════════════╡
│ car_0  ┆ 2023-01-01T00:00:00        ┆ 31.619685 ┆ -88.849536 ┆ null          │
│ car_0  ┆ 2023-01-01T00:19:16.934859 ┆ 31.619685 ┆ -88.849536 ┆ low_fuel      │
│ car_0  ┆ 2023-01-01T00:30:00        ┆ 31.628011 ┆ -88.855186 ┆ null          │
│ car_0  ┆ 2023-01-01T00:40:31.791731 ┆ 31.628011 ┆ -88.855186 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01T00:33:56.583517 ┆ 31.628011 ┆ -88.855186 ┆ null          │
└────────┴────────────────────────────┴───────────┴────────────┴───────────────┘


## Limpieza de datos

### Timestamp

Convierte el `timestamp` que actualmente es string a formato de tiempo en polars

In [52]:
data = data.with_columns(
    pl.col("timestamp").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S.%f", strict=False).alias("timestamp")
)

data = data.collect()
print(data.head())

/var/folders/19/9kqb3cd921sd204qnh8grxs40000gn/T/ipykernel_52942/3659594967.py:2: ChronoFormatWarning: Detected the pattern `.%f` in the chrono format string. This pattern should not be used to parse values after a decimal point. Use `%.f` instead. See the full specification: https://docs.rs/chrono/latest/chrono/format/strftime
  pl.col("timestamp").str.strptime(pl.Datetime, format="%Y-%m-%dT%H:%M:%S.%f", strict=False).alias("timestamp")


shape: (5, 5)
┌────────┬────────────────────────────┬───────────┬────────────┬───────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude  ┆ notification  │
│ ---    ┆ ---                        ┆ ---       ┆ ---        ┆ ---           │
│ str    ┆ datetime[μs]               ┆ f64       ┆ f64        ┆ str           │
╞════════╪════════════════════════════╪═══════════╪════════════╪═══════════════╡
│ car_0  ┆ null                       ┆ 31.619685 ┆ -88.849536 ┆ null          │
│ car_0  ┆ 2023-01-01 00:19:16.000934 ┆ 31.619685 ┆ -88.849536 ┆ low_fuel      │
│ car_0  ┆ null                       ┆ 31.628011 ┆ -88.855186 ┆ null          │
│ car_0  ┆ 2023-01-01 00:40:31.000791 ┆ 31.628011 ┆ -88.855186 ┆ tire_pressure │
│ car_0  ┆ 2023-01-01 00:33:56.000583 ┆ 31.628011 ┆ -88.855186 ┆ null          │
└────────┴────────────────────────────┴───────────┴────────────┴───────────────┘


### Ingenieria de variables

Dado que va a entrar a un modelo de machine learning es encesario que todas las variables sean numericas, y esten en formnato tidy. Cada observacion en una fila, y cada variable en una columna. Por lo tanto se decidio crear estadisticos y agregar los datos a intervalos uniformes de `x` minutos.  

Por ejemplo, colapsar toda la informacion que ocurrion en el intervalo, como el numero de notificaciones en esos 5 minutos, el promedio entre notificaciones, y el tipo de notificaciones.

Existen varias formas de hacer esto, puedes hacerlo con `group_by` primero para crear las nuevas variables, o `group_by` (`rolling`, `dynamic`) usando operaciones sobre listas. Utiliza claude o chat_gpt

1. Crea una nueva variable que compute la diferencia de tiempo entre notificaciones del mismo vehiculo. Piensa como lo vas a hacer. Llama a esta variable `notification_time`
   


In [53]:
data = data.with_columns(
    pl.col("timestamp")
    .diff()  
    .cast(pl.Int64)
    .over("car_id")
    .alias("notification_time")
)

print(data.head())

shape: (5, 6)
┌────────┬────────────────────────────┬───────────┬────────────┬───────────────┬───────────────────┐
│ car_id ┆ timestamp                  ┆ latitude  ┆ longitude  ┆ notification  ┆ notification_time │
│ ---    ┆ ---                        ┆ ---       ┆ ---        ┆ ---           ┆ ---               │
│ str    ┆ datetime[μs]               ┆ f64       ┆ f64        ┆ str           ┆ i64               │
╞════════╪════════════════════════════╪═══════════╪════════════╪═══════════════╪═══════════════════╡
│ car_0  ┆ null                       ┆ 31.619685 ┆ -88.849536 ┆ null          ┆ null              │
│ car_0  ┆ 2023-01-01 00:19:16.000934 ┆ 31.619685 ┆ -88.849536 ┆ low_fuel      ┆ null              │
│ car_0  ┆ null                       ┆ 31.628011 ┆ -88.855186 ┆ null          ┆ null              │
│ car_0  ┆ 2023-01-01 00:40:31.000791 ┆ 31.628011 ┆ -88.855186 ┆ tire_pressure ┆ null              │
│ car_0  ┆ 2023-01-01 00:33:56.000583 ┆ 31.628011 ┆ -88.855186 ┆ null        

2. Crea una nueva variable que compute la distancia que viajo el vehiculo desde la ultima notificacion. Llamala `distance`

In [54]:
from geopy import distance

def dist(prev_lat, prev_lon, lat, lon):
    if prev_lat is None or prev_lon is None:
        return None  
    
    aux = distance.geodesic((prev_lat, prev_lon), (lat, lon)).meters

    return aux

data = data.with_columns([
    pl.col("latitude").shift(1).over("car_id").alias("prev_latitude"),
    pl.col("longitude").shift(1).over("car_id").alias("prev_longitude"),]).with_columns(
        pl.struct(["prev_latitude", "prev_longitude", "latitude", "longitude"])
        .map_elements(lambda row: dist(row["prev_latitude"], row["prev_longitude"], row["latitude"], row["longitude"]))
        .alias("distance")
)

print(data.head())

/var/folders/19/9kqb3cd921sd204qnh8grxs40000gn/T/ipykernel_52942/3240703342.py:11: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  data = data.with_columns([


shape: (5, 9)
┌────────┬────────────┬───────────┬────────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ car_id ┆ timestamp  ┆ latitude  ┆ longitude  ┆ … ┆ notificat ┆ prev_lati ┆ prev_long ┆ distance  │
│ ---    ┆ ---        ┆ ---       ┆ ---        ┆   ┆ ion_time  ┆ tude      ┆ itude     ┆ ---       │
│ str    ┆ datetime[μ ┆ f64       ┆ f64        ┆   ┆ ---       ┆ ---       ┆ ---       ┆ f64       │
│        ┆ s]         ┆           ┆            ┆   ┆ i64       ┆ f64       ┆ f64       ┆           │
╞════════╪════════════╪═══════════╪════════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ car_0  ┆ null       ┆ 31.619685 ┆ -88.849536 ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ car_0  ┆ 2023-01-01 ┆ 31.619685 ┆ -88.849536 ┆ … ┆ null      ┆ 31.619685 ┆ -88.84953 ┆ 0.0       │
│        ┆ 00:19:16.0 ┆           ┆            ┆   ┆           ┆           ┆ 6         ┆           │
│        ┆ 00934      ┆           ┆            ┆   ┆           ┆           ┆ 

3. Crea intervalos de `x` minutos por carro. Como el numero de notificaciones en esos intervalos no es uniforme tienes que buscar funciones de polars especificas, pero ademas tienen que ser por vehiculo, pues tienen que ser del mismo. Revisa las funciones de `group_by` `dynamic` y `rolling`.
   1. Computa la media, mediana, varianza, max y min de `notification_time` los intervalos de `x` minutos
   2. Computa la media, mediana, varianza, max y min de `distance`


In [55]:
data = data.lazy()
data = (
    data
    .with_columns([
        pl.col("car_id"),
        pl.col("timestamp"),
        pl.col("notification_time")
        .fill_null(0) 
        .cast(pl.Float64),  
        pl.col("distance")
        .fill_null(0) 
        .cast(pl.Float64),
    ])
    .group_by_dynamic(
        index_column="timestamp",
        by="car_id",  
        every="5m",
        closed="left"  
    )
    .agg([
        pl.col("notification_time").mean().alias("notification_time_mean"),
        pl.col("notification_time").median().alias("notification_time_median"),
        pl.col("notification_time").var().alias("notification_time_variance"),
        pl.col("notification_time").max().alias("notification_time_max"),
        pl.col("notification_time").min().alias("notification_time_min"),
        pl.col("distance").mean().alias("distance_mean"),
        pl.col("distance").median().alias("distance_median"),
        pl.col("distance").var().alias("distance_variance"),
        pl.col("distance").max().alias("distance_max"),
        pl.col("distance").min().alias("distance_min"),
    ])
)

/var/folders/19/9kqb3cd921sd204qnh8grxs40000gn/T/ipykernel_52942/2416787850.py:3: DeprecationWarning: The argument `by` for `LazyFrame.group_by_dynamic` is deprecated. It has been renamed to `group_by`.
  data


In [57]:
data.collect().head()

ComputeError: null values in dynamic group_by not supported, fill nulls.